In [1]:
import logging
import pathlib
import sys
from typing import Optional

import numpy as np
import pandas as pd
import toml
from copairs.map import run_pipeline
from pycytominer import feature_select

# imports src
sys.path.append("../")
from src import utils

# setting up logger
logging.basicConfig(
    filename="map_analysis_testing.log",
    level=logging.DEBUG,
    format="%(levelname)s:%(asctime)s:%(name)s:%(message)s",
)

## Helper functions
Set of helper functions to help out throughout the notebook

In [2]:
## Helper function


def shuffle_meta_labels(
    dataset: pd.DataFrame, target_col: str, seed: Optional[int] = 0
) -> pd.DataFrame:
    """shuffles labels or values within a single selected column

    Parameters
    ----------
    dataset : pd.DataFrame
        dataframe containing the dataset

    target_col : str
        Column to select in order to conduct the shuffling

    seed : int
        setting random seed

    Returns
    -------
    pd.DataFrame
        shuffled dataset

    Raises
    ------
    TypeError
        raised if incorrect types are provided
    """
    # setting seed
    np.random.seed(seed)

    # type checking
    if not isinstance(target_col, str):
        raise TypeError("'target_col' must be a string type")
    if not isinstance(dataset, pd.DataFrame):
        raise TypeError("'dataset' must be a pandas dataframe")

    # selecting column, shuffle values within column, add to dataframe
    dataset[target_col] = np.random.permutation(dataset[target_col].values)
    return dataset


def shuffle_features(feature_vals: np.array, seed: Optional[int] = 0) -> np.array:
    """suffles all values within feature space

    Parameters
    ----------
    feature_vals : np.array
        shuffled

    seed : Optional[int]
        setting random seed

    Returns
    -------
    np.array
        Returns shuffled values within the feature space

    Raises
    ------
    TypeError
        Raised if a numpy array is not provided
    """
    # setting seed
    np.random.seed(seed)

    # shuffle given array
    if not isinstance(feature_vals, np.ndarray):
        raise TypeError("'feature_vals' must be a numpy array")
    if feature_vals.ndim != 2:
        raise TypeError("'feature_vals' must be a 2x2 matrix")

    # creating a copy for feature vales to prevent overwriting of global variables
    feature_vals = np.copy(feature_vals)

    # shuffling feature space
    n_cols = feature_vals.shape[1]
    for col_idx in range(0, n_cols):
        # selecting column, shuffle, and update:
        feature_vals[:, col_idx] = np.random.permutation(feature_vals[:, col_idx])

    return feature_vals

## Setting up Paths and loading data

In [3]:
# load in the treatment groups
ground_truth = pathlib.Path(
    "../../4.sc_Morphology_Neural_Network_MLP_Model/MLP_utils/ground_truth.toml"
).resolve(strict=True)
# load in the ground truth
ground_truth = toml.load(ground_truth)
apoptosis_ground_truth = ground_truth["Apoptosis"]["apoptosis_groups_list"]
pyroptosis_ground_truth = ground_truth["Pyroptosis"]["pyroptosis_groups_list"]
control_ground_truth = ground_truth["Healthy"]["healthy_groups_list"]

In [4]:
single_cell_data = pathlib.Path(
    f"../../data/PBMC_preprocessed_sc_norm_aggregated.parquet"
).resolve(strict=True)
df = pd.read_parquet(single_cell_data)

In [5]:
# out paths
map_out_dir = pathlib.Path("../data/processed/mAP_scores/morphology/")
map_out_dir.mkdir(exist_ok=True, parents=True)

# regular data output
# saving to csv
regular_feat_map_path = pathlib.Path(map_out_dir / "mAP_scores_regular_treatment.csv")

# shuffled data output
shuffled_feat_map_path = pathlib.Path(
    map_out_dir / "mAP_scores_shuffled_class_treatment.csv"
)

# shuffled feature space output
shuffled_feat_space_map_path = pathlib.Path(
    map_out_dir / "mAP_scores_shuffled_feature_space_treatment.csv"
)

### Clean up data

In [6]:
# add apoptosis, pyroptosis and healthy columns to dataframe
df["Apoptosis"] = df.apply(
    lambda row: row["oneb_Metadata_Treatment_Dose_Inhibitor_Dose"]
    in apoptosis_ground_truth,
    axis=1,
)
df["Pyroptosis"] = df.apply(
    lambda row: row["oneb_Metadata_Treatment_Dose_Inhibitor_Dose"]
    in pyroptosis_ground_truth,
    axis=1,
)
df["Control"] = df.apply(
    lambda row: row["oneb_Metadata_Treatment_Dose_Inhibitor_Dose"]
    in control_ground_truth,
    axis=1,
)

# merge apoptosis, pyroptosis, and healthy columns into one column
df["Metadata_labels"] = df.apply(
    lambda row: "Apoptosis"
    if row["Apoptosis"]
    else "Pyroptosis"
    if row["Pyroptosis"]
    else "Control",
    axis=1,
)
# # drop apoptosis, pyroptosis, and healthy columns
df.drop(columns=["Apoptosis", "Pyroptosis", "Control"], inplace=True)

df["oneb_Metadata_Treatment_Dose_Inhibitor_Dose"].unique()
# replace values in the oneb_Metadata_Treatment_Dose_Inhibitor_Dose column
df["oneb_Metadata_Treatment_Dose_Inhibitor_Dose"] = df[
    "oneb_Metadata_Treatment_Dose_Inhibitor_Dose"
].replace(
    "Flagellin_0.100_ug_per_ml_DMSO_0.000_%", "Flagellin_0.100_ug_per_ml_DMSO_0.025_%"
)
df["oneb_Metadata_Treatment_Dose_Inhibitor_Dose"] = df[
    "oneb_Metadata_Treatment_Dose_Inhibitor_Dose"
].replace("Flagellin_1.000_0_DMSO_0.025_%", "Flagellin_1.000_ug_per_ml_DMSO_0.025_%")
df["oneb_Metadata_Treatment_Dose_Inhibitor_Dose"] = df[
    "oneb_Metadata_Treatment_Dose_Inhibitor_Dose"
].replace(
    "Flagellin_1.000_ug_per_ml_DMSO_0.000_%", "Flagellin_1.000_ug_per_ml_DMSO_0.025_%"
)
df["oneb_Metadata_Treatment_Dose_Inhibitor_Dose"] = df[
    "oneb_Metadata_Treatment_Dose_Inhibitor_Dose"
].replace("media_ctr_0.0_0_Media_0_0", "Media")
df["oneb_Metadata_Treatment_Dose_Inhibitor_Dose"] = df[
    "oneb_Metadata_Treatment_Dose_Inhibitor_Dose"
].replace("media_ctr_0.0_0_Media_ctr_0.0_0", "Media")
df["oneb_Metadata_Treatment_Dose_Inhibitor_Dose"] = df[
    "oneb_Metadata_Treatment_Dose_Inhibitor_Dose"
].replace(
    "Flagellin_1.000_0_Disulfiram_1.000_uM",
    "Flagellin_1.000_ug_per_ml_Disulfiram_1.000_uM",
)
len(df["oneb_Metadata_Treatment_Dose_Inhibitor_Dose"].unique())

37

In [7]:
# output directories
map_out_dir = pathlib.Path("../data/processed/mAP_scores/")
map_out_dir.mkdir(parents=True, exist_ok=True)

### mAP Pipeline Parameters

The null size needs to be determined for the mAP pipeline. This is the size of the null class that is used to determine the mAP score.

In [8]:
tmp = (
    df.groupby(["oneb_Metadata_Treatment_Dose_Inhibitor_Dose"])
    .count()
    .reset_index()[["Metadata_Well", "oneb_Metadata_Treatment_Dose_Inhibitor_Dose"]]
)
# get the counts of each oneb_Metadata_Treatment_Dose_Inhibitor_Dose
min_count = tmp["Metadata_Well"].min()
print(min_count)

4


In [9]:
pos_sameby = [
    "oneb_Metadata_Treatment_Dose_Inhibitor_Dose",
]
pos_diffby = ["Metadata_Well"]

neg_sameby = []
neg_diffby = ["oneb_Metadata_Treatment_Dose_Inhibitor_Dose"]

null_size = min_count
batch_size = 1

# number of resampling
n_resamples = 10

### mAP analysis for non-shuffled data

In [10]:
# This will generated 100 values [0..100] as seed values
# This will occur per phenotype

# spliting metadata and raw feature values
logging.info("splitting data set into metadata and raw feature values")
df_meta, df_feats = utils.split_data(df)
df_feats = np.array(df_feats)

# execute pipeline on negative control with training dataset with cp features
try:
    # execute pipeline on negative control with trianing dataset with cp features
    logging.info(f"Running pipeline on CP features using phenotype")
    result = run_pipeline(
        meta=df_meta,
        feats=df_feats,
        pos_sameby=pos_sameby,
        pos_diffby=pos_diffby,
        neg_sameby=neg_sameby,
        neg_diffby=neg_diffby,
        batch_size=batch_size,
        null_size=null_size,
    )

    # adding columns
    result["shuffled"] = "non-shuffled"


except ZeroDivisionError as e:
    logging.warning(f"{e} captured on phenotye:. Skipping")
# concatenating all datasets
result.to_csv(regular_feat_map_path, index=False)
result.head()

  0%|          | 0/253 [00:00<?, ?it/s]

  0%|          | 0/11528 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

,Metadata_Well,oneb_Metadata_Treatment_Dose_Inhibitor_Dose,Metadata_labels,average_precision,p_value,n_pos_pairs,n_total_pairs,shuffled
0,B02,LPS_0.010_ug_per_ml_DMSO_0.025_%,Pyroptosis,0.072612,0.2,3,153,non-shuffled
1,B03,LPS_0.010_ug_per_ml_DMSO_0.025_%,Pyroptosis,0.063540,0.2,3,153,non-shuffled
2,B04,LPS_Nigericin_100.000_ug_per_ml_1.000_uM_DMSO_...,Pyroptosis,0.497076,0.2,3,153,non-shuffled
3,B05,LPS_Nigericin_100.000_ug_per_ml_1.000_uM_DMSO_...,Pyroptosis,1.000000,0.2,3,153,non-shuffled
4,B06,DMSO_0.100_%_DMSO_0.025_%,Control,0.045209,1.0,7,153,non-shuffled


### mAP analysis for shuffled data (Phenotype)

In [11]:
logging.info("Running mAP pipeline with shuffled phenotype labeled data")
seed = 0
# running process
# for loop selects one single phenotype
# then splits the data into metadata and raw feature values
# two different groups that contains 3 splits caused by the types of features

# This will generated 100 values [0..100] as seed values
# splitting metadata labeled shuffled data
logging.info("splitting shuffled data set into metadata and raw feature values")
df = shuffle_meta_labels(
    dataset=df, target_col="oneb_Metadata_Treatment_Dose_Inhibitor_Dose", seed=seed
)
(
    df_meta,
    df_feats,
) = utils.split_data(df)


df_feats = np.array(df_feats)

try:
    # execute pipeline on negative control with trianing dataset with cp features
    logging.info(
        f"Running pipeline on CP features using  phenotype, data is shuffled by phenoptype labels"
    )
    shuffled_result = run_pipeline(
        meta=df_meta,
        feats=df_feats,
        pos_sameby=pos_sameby,
        pos_diffby=pos_diffby,
        neg_sameby=neg_sameby,
        neg_diffby=neg_diffby,
        batch_size=batch_size,
        null_size=null_size,
    )

    # adding shuffle label column
    shuffled_result["shuffled"] = "phenotype_shuffled"


except ZeroDivisionError as e:
    logging.warning(f"{e} captured on phenotye: Skipping")

# saving to csv
shuffled_result.to_csv(shuffled_feat_map_path, index=False)
shuffled_result

  0%|          | 0/253 [00:00<?, ?it/s]

  0%|          | 0/11528 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

,Metadata_Well,oneb_Metadata_Treatment_Dose_Inhibitor_Dose,Metadata_labels,average_precision,p_value,n_pos_pairs,n_total_pairs,shuffled
0,B02,Thapsigargin_1.000_uM_DMSO_0.025_%,Pyroptosis,0.021463,0.8,3,153,phenotype_shuffled
1,B03,LPS_100.000_ug_per_ml_DMSO_0.025_%,Pyroptosis,0.053956,0.4,3,153,phenotype_shuffled
2,B04,Media,Pyroptosis,0.043059,0.8,5,153,phenotype_shuffled
3,B05,DMSO_0.100_%_Z-VAD-FMK_100.000_uM,Pyroptosis,0.138639,0.2,3,153,phenotype_shuffled
4,B06,LPS_Nigericin_100.000_ug_per_ml_10.000_uM_DMSO...,Control,0.039502,0.4,3,153,phenotype_shuffled
...,...,...,...,...,...,...,...,...
149,O08,LPS_Nigericin_100.000_ug_per_ml_1.000_uM_DMSO_...,Control,0.015444,1.0,3,153,phenotype_shuffled
150,O09,Thapsigargin_1.000_uM_DMSO_0.025_%,Control,0.050063,0.4,3,153,phenotype_shuffled
151,O10,LPS_10.000_ug_per_ml_Disulfiram_2.500_uM,Control,0.035600,0.6,3,153,phenotype_shuffled
152,O11,LPS_Nigericin_1.000_ug_per_ml_3.000_uM_DMSO_0....,Control,0.032986,0.6,3,153,phenotype_shuffled


### mAP analysis for shuffled data (Feature space)

In [12]:
seed = 0


# split the shuffled dataset
# spliting metadata and raw feature values
logging.info("splitting shuffled data set into metadata and raw feature values")
(
    df_meta,
    df_feats,
) = utils.split_data(df)

df_feats = np.array(df_feats)


# shuffling the features, this will overwrite the generated feature space from above with the shuffled one
df_feats = shuffle_features(feature_vals=df_feats, seed=seed)


try:
    # execute pipeline on negative control with trianing dataset with cp features
    logging.info(
        f"Running pipeline on CP features using phenotype, feature space is shuffled"
    )
    shuffled_feat_map = run_pipeline(
        meta=df_meta,
        feats=df_feats,
        pos_sameby=pos_sameby,
        pos_diffby=pos_diffby,
        neg_sameby=neg_sameby,
        neg_diffby=neg_diffby,
        batch_size=batch_size,
        null_size=null_size,
    )

    # adding shuffle label column
    shuffled_feat_map["shuffled"] = "features_shuffled"


except ZeroDivisionError as e:
    logging.warning(f"{e} captured on phenotype:  Skipping")


# saving to csv
shuffled_feat_map.to_csv(shuffled_feat_space_map_path, index=False)
shuffled_feat_map

  0%|          | 0/253 [00:00<?, ?it/s]

  0%|          | 0/11528 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

,Metadata_Well,oneb_Metadata_Treatment_Dose_Inhibitor_Dose,Metadata_labels,average_precision,p_value,n_pos_pairs,n_total_pairs,shuffled
0,B02,Thapsigargin_1.000_uM_DMSO_0.025_%,Pyroptosis,0.035841,0.6,3,153,features_shuffled
1,B03,LPS_100.000_ug_per_ml_DMSO_0.025_%,Pyroptosis,0.017741,1.0,3,153,features_shuffled
2,B04,Media,Pyroptosis,0.046681,0.4,5,153,features_shuffled
3,B05,DMSO_0.100_%_Z-VAD-FMK_100.000_uM,Pyroptosis,0.023791,0.8,3,153,features_shuffled
4,B06,LPS_Nigericin_100.000_ug_per_ml_10.000_uM_DMSO...,Control,0.040056,0.4,3,153,features_shuffled
...,...,...,...,...,...,...,...,...
149,O08,LPS_Nigericin_100.000_ug_per_ml_1.000_uM_DMSO_...,Control,0.032195,0.6,3,153,features_shuffled
150,O09,Thapsigargin_1.000_uM_DMSO_0.025_%,Control,0.016167,1.0,3,153,features_shuffled
151,O10,LPS_10.000_ug_per_ml_Disulfiram_2.500_uM,Control,0.038991,0.4,3,153,features_shuffled
152,O11,LPS_Nigericin_1.000_ug_per_ml_3.000_uM_DMSO_0....,Control,0.021037,0.8,3,153,features_shuffled
